## Load modules

In [ ]:
addprocs(int(CPU_CORES / 2))

using DiscreteValueIteration
using SCAs, SCAIterators, SCAConst
using GridInterpolations

mdp = SCA()

## Check size of MDP

In [ ]:
function getBytes(x)
   total = 0;
   fieldNames = typeof(x).names;
   if fieldNames == ()
      return sizeof(x);
   else
     for fieldName in fieldNames
        total += getBytes(getfield(x,fieldName));
     end
     return total;
   end
end

println(typeof(mdp), " takes up ", getBytes(mdp) / 1000.0, " kB")

## Informal validation of state and action iterators and transition function

In [ ]:
testActions = ActionIterator(Actions)
actionCount = 0
for action in testActions
    actionCount += 1
    println("action #", actionCount, ": ", typeof(action), " = ", action)
end # for action
println("nActions = ", NActions)
println("actionCount = ", actionCount)

testStates = StateIterator(Xs, Ys, Bearings, Speeds, Speeds)
stateCount = 0
for state in testStates
    stateCount += 1
    if stateCount == 1 || stateCount == NStates
        println("state #", stateCount, ": ", typeof(state), " = ", state)
    end # if
end # for state
println("nStates = ", NStates)
println("stateCount = ", stateCount)

In [ ]:
newStates, probs = nextStates(mdp, State(1.0,1.0,1.0,10.0,10.0,false), Action(:straight, :straight))

## Serial solution

In [ ]:
maxIterations = 100
tolerance = 1e-3
gs = true
includeV = true
includeQ = true
includeA = false
solver = SerialSolver(
    maxIterations = maxIterations,
    tolerance = tolerance,
    gaussSiedel = gs,
    includeV = includeV,
    includeQ = includeQ,
    includeA = includeA)

In [ ]:
policy = solve(solver, mdp, verbose = true)

## Parallel solution

In [ ]:
numProcs = int(CPU_CORES / 2)
stateOrder = {1:int64(floor(NStates / 2)), int64(floor(NStates / 2)) + 1:NStates}
solver = ParallelSolver(
    numProcs,
    stateOrder = stateOrder,
    maxIterations = maxIterations,
    tolerance = tolerance,
    gaussSiedel = gs,
    includeV = includeV,
    includeQ = includeQ,
    includeA = includeA)

In [ ]:
policy = solve(solver, mdp, verbose = true)